In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
from IPython.display import HTML, display , Markdown



In [ ]:
base_path = "E:\\G- 1539 Blue Cat Seagate 500 GB\\Drive 2\\upgrad\\lending\\"
lending_file = base_path + "loan.zip"


# # Read File 

In [ ]:
pd_lend_df = pd.read_csv(lending_file)
pd_lend_df.info()

# # Find and Drop empty columns 

In [ ]:
pd_lend_df.dropna(axis = 1, how = 'all' ,inplace = True)
pd_lend_df.info()


# # Find and Drop Columns with unique value

In [ ]:
pd_lend_df=pd_lend_df.loc[:,pd_lend_df.nunique()>1]
pd_lend_df.info()

# # loan_status main target column  with object type, a quick look

In [ ]:
pd_lend_df['loan_status'].describe()

In [ ]:
#Find unique values and keep relevant
pd_lend_df['loan_status'] .unique()

In [ ]:
# loan_status of current means ongoing loan and is very small no. , dropping such rows and change type
pd_lend_df = pd_lend_df[~(pd_lend_df['loan_status'] == "Current")]
pd_lend_df['loan_status'] = pd_lend_df['loan_status'].astype('category')
# out_prncp_inv and out_prncp are only related to current loans, dropping them
pd_lend_df.drop(columns = ['out_prncp_inv','out_prncp'], inplace = True)

# # Drop columns with < 30% non-null data

In [ ]:
# From info, it is seen that there are 2 columns with , 30% non-null data , drop them
pd_lend_df.drop(columns = ['next_pymnt_d','mths_since_last_record'], inplace = True)

# # Cleanup text columns

In [ ]:
pd_lend_df.describe(include ='object')

In [ ]:
# term column should have numeric data , find all text and replace and change dtype
print(len(pd_lend_df[pd_lend_df['term'].str.contains("months") == True]))

In [ ]:
pd_lend_df['term'] = pd_lend_df['term'].str.replace("months" ,"")
pd_lend_df['term'] = pd_lend_df['term'].astype('category')


In [ ]:
# int_rate seems to have % symbol, find out , replace and change type
print(len(pd_lend_df[pd_lend_df['int_rate'].str.contains("%") == True]))

In [ ]:
pd_lend_df['int_rate'] = pd_lend_df['int_rate'].str.replace("%" ,"")
pd_lend_df['int_rate'] = pd_lend_df['int_rate'].astype('float')

# # # # Change type to date and date-time as applicable

In [ ]:
pd_lend_df['issue_d'] = pd.to_datetime(pd_lend_df['issue_d'],format = '%b-%y' )
pd_lend_df['last_pymnt_d'] = pd.to_datetime(pd_lend_df['last_pymnt_d'],format = '%b-%y' )
pd_lend_df['last_credit_pull_d'] = pd.to_datetime(pd_lend_df['last_credit_pull_d'],format = '%b-%y' )
pd_lend_df['earliest_cr_line'] = pd.to_datetime(pd_lend_df['earliest_cr_line'],format = '%b-%y' )

In [ ]:
# relook at object columns
pd_lend_df.describe(include ='object')

In [ ]:
# revol_util also have % symbol, drop that and update type , seems this is only such column now
print(len(pd_lend_df[pd_lend_df['revol_util'].str.contains("%") == True]))

In [ ]:
pd_lend_df['revol_util'] = pd_lend_df['revol_util'].str.replace("%" ,"")
pd_lend_df['revol_util'] = pd_lend_df['revol_util'].astype('float')

In [ ]:
# emp_length should have values between 0 to 10 , cleanup text and change type
print(len(pd_lend_df[pd_lend_df['emp_length'].str.contains("years") == True]))
print(pd_lend_df['emp_length'].unique())

In [ ]:
pd_lend_df['emp_length'] = pd_lend_df['emp_length'].str.replace("< 1" ,"0")
pd_lend_df['emp_length'] = pd_lend_df['emp_length'].str.replace("+" ,"")
pd_lend_df['emp_length'] = pd_lend_df['emp_length'].str.replace("years" ,"")
pd_lend_df['emp_length'] = pd_lend_df['emp_length'].str.replace("year" ,"")
pd_lend_df['emp_length'] = pd_lend_df['emp_length'].str.replace(" " ,"")
print(pd_lend_df['emp_length'].unique())
#pd_lend_df.describe(include ='object')



In [ ]:
# nan emp_length mostly means 0 , lets analyze  
fig = px.pie(pd_lend_df, names='emp_length')
fig.show()


In [ ]:
# 0 is second largest value , lets replace nan with 0  and change column type
pd_lend_df['emp_length'] = pd_lend_df['emp_length'].fillna(0)
pd_lend_df['emp_length']  = pd_lend_df['emp_length'].astype('category')

# # # # # URL column

In [ ]:
#url column is giving clutterred view when described, lets analyze
import urllib.parse as urlparse
pd_lend_df['protocol'],pd_lend_df['domain'],pd_lend_df['path'],pd_lend_df['query'],pd_lend_df['fragment'] = zip(*pd_lend_df['url'].map(urlparse.urlsplit))


In [ ]:
pd_lend_df[['protocol','domain','path','query' ,'fragment']].describe()

In [ ]:
#drop url and components with unique values and replace loan-id text and make it integer
pd_lend_df.drop(columns = ['protocol','domain','path','fragment','url'] ,inplace = True)
pd_lend_df['query'] = pd_lend_df['query'].str.replace("loan_id=","")
pd_lend_df['query']  = pd_lend_df['query'].astype(int)

In [ ]:
# check if id and loan id are same
pd_lend_df['diffce'] = pd_lend_df['query'] - pd_lend_df['id']
pd_lend_df['diffce'].describe()

In [ ]:
# drop calculation column created above
pd_lend_df.drop(columns=['diffce'], inplace = True)

In [ ]:
# drop query column
pd_lend_df.drop(columns=['query'], inplace = True)

In [ ]:
pd_lend_df.describe(include ='object')

In [ ]:
# Seems some object columns can be converted to category ,
pd_lend_df['grade'] = pd_lend_df['grade'].astype('category')
pd_lend_df['home_ownership'] = pd_lend_df['home_ownership'].astype('category')
pd_lend_df['purpose'] = pd_lend_df['purpose'].astype('category')
pd_lend_df['addr_state'] = pd_lend_df['addr_state'].astype('category')
pd_lend_df['verification_status'] = pd_lend_df['verification_status'].astype('category')


In [ ]:
#also title and purpose seem to have similar values , title having lot unique values and  purpose with few categories, drop it 
# from data dictionary , we know that emp_title is mix of employer name and title , it has lot of quniue values, lets drop
pd_lend_df.drop(columns=['emp_title','title'], inplace = True)

In [ ]:
pd_lend_df.describe(include ='object')

# # # # # Cleanup desc column

In [ ]:
pd_lend_df['desc']

In [ ]:
# seems too long text in desc column
pd_lend_df['desc'].str.len().describe()

In [ ]:
#It seems long text with purpose plus some info. , might not be relevant for our analysis , lets drop to save space
pd_lend_df.drop(columns=['desc'], inplace = True)

# # # # # zip_code clean up

In [ ]:
# zip code should have 3 digit nos, remove xxx and find relationship addr state
pd_lend_df['zip_code'] = pd_lend_df['zip_code'].str.replace("xx" ,"")
pd_lend_df.groupby('addr_state')['zip_code'].describe()

In [ ]:
# none of the zip codes have enough records ( > 30%) , dropping it
pd_lend_df.drop(columns=['zip_code'], inplace = True)
pd_lend_df.describe(include ='object')

In [ ]:
# sub grade and grade are related and sub_grade is grade + number , lets remove text
pd_lend_df['sub_grade'] = pd_lend_df['sub_grade'].str[1:]
pd_lend_df['sub_grade'] = pd_lend_df['sub_grade'].astype('category')


# # # # Drop more unnecessary columns

In [ ]:
pd_lend_df.drop(columns=['funded_amnt_inv','total_pymnt_inv','total_pymnt','pub_rec_bankruptcies'], inplace = True)
                       # 'mths_since_last_delinq','total_rec_late_fee','recoveries','total_rec_late_fee','collection_recovery_fee']

# for now keep -- these columns are post charged off -- 'mths_since_last_delinq','total_rec_late_fee','recoveries','total_rec_late_fee','collection_recovery_fee'
# _inv is for investors contribution, it is highly correlated with other columns with similar numeric data not relevant for this exercise at this moment
# total_pymnt = total_rec_prncp + total_rec_int , hence dropping                      
# pub_rec_bankruptcies kind of same as pub_rec and comes much later

In [ ]:
# relook at info
pd_lend_df.info()

# Quick Look and cleanup of numeric columns

In [ ]:
# analyze numeric columns
pd_lend_df.describe()

In [ ]:
len(pd_lend_df['id'].unique())

In [ ]:
len(pd_lend_df['member_id'].unique())

In [ ]:
# both id and member_id are unique , member_id is actual applicant id , so lets keep it as index  and drop id
pd_lend_df.drop(columns = ['id'] , inplace = True)

In [ ]:
pd_lend_df.set_index('member_id' , inplace = True)

In [ ]:
# delinq_2yrs , inq_last_6mths , pub_rec  , mths_since_last_delinq  might be categorical and/or int columns
pd_lend_df['delinq_2yrs'].unique()

In [ ]:
pd_lend_df['inq_last_6mths'].unique()

In [ ]:
pd_lend_df['pub_rec'].unique()

# # # # # # Treat Na values in mths_since_last_delinq

In [ ]:
# mths_since_last_delinq nan means either no records or no delinquency
pd_lend_df['mths_since_last_delinq'] = pd_lend_df['mths_since_last_delinq'].fillna(0)

In [ ]:
pd_lend_df['delinq_2yrs'] = pd_lend_df['delinq_2yrs'].astype(int).astype('category')
pd_lend_df['inq_last_6mths'] = pd_lend_df['inq_last_6mths'].astype(int).astype('category')
pd_lend_df['pub_rec'] = pd_lend_df['pub_rec'].astype(int).astype('category')
pd_lend_df['mths_since_last_delinq'] = pd_lend_df['mths_since_last_delinq'].astype('int')
#fnor lets make it categorical , later see how to handle nan values
pd_lend_df.describe()

# # # # # Not sure of floating point precision, round it to 2 for now

In [ ]:
df_num = pd_lend_df.select_dtypes(include=[float]).columns
df_num

In [ ]:
for each_float_column in df_num:
    pd_lend_df[each_float_column] = round(pd_lend_df[each_float_column],2)
pd_lend_df.describe()

# Find raw correlations

In [ ]:
corr = pd_lend_df.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

# Conclusion
Except loan_amnt, funded_amnt , installment , total_rec_prncp , total_rec_int which are highly related , rest of the columns do not seem to  strong positive and/or negative correlation.
Most of the negative correlations are very weak(no correlation) and positive correlations are either very weak(no correlation)  or weak to medium.

# Univariate Analysis

# # Numeric Columns

In [ ]:
#For many numeric variables, box plots is needed, lets modularize it using a function
def plot_box_plot_univariate(univariate_col_y):
    fig = px.box(pd_lend_df, y=univariate_col_y , title = "box plot for " + univariate_col_y)
    fig.update_traces(quartilemethod="linear") # or "inclusive", or "linear" by default
    fig.show()

In [ ]:
df_num = pd_lend_df.select_dtypes(include=[np.number]).columns
for each_numeric_column in df_num:
    plot_box_plot_univariate(each_numeric_column)

# # # Outliar treatment

In [ ]:
# # # # it seems there are some columns that have outliars,
#for now , among input columns annual_inc is imp. , lets treat outliars
# rest we will see later

In [ ]:
#annual income seems to have some outliars  , lets analyze before treating them above  ~1.5* q3
pd_annual_inc_out = pd_lend_df[pd_lend_df['annual_inc'] >= 250000][['loan_amnt','loan_status','annual_inc','emp_length']].copy()
#pd_annual_inc_out = pd_annual_inc_out[pd_annual_inc_out['loan_status'] == 'Charged Off']
#print(len(pd_annual_inc_out) , print(len(pd_annual_inc_out[pd_annual_inc_out['loan_status'] == 'Charged Off'])))
pd_annual_inc_out

In [ ]:
print(len(pd_annual_inc_out[pd_annual_inc_out['loan_status'] == 'Charged Off']))

In [ ]:
# for >0.25M income group , total no. of loans and the loan amonuts are not high and only few loans Charged off , hence replacing outliars with median
#Also adding a column to identify if outliars are adjusted
pd_lend_df['annual_inc_outliar_adjust'] = np.where(pd_lend_df['annual_inc'] >= 250000, pd_lend_df['annual_inc'],0)
pd_lend_df['annual_inc'] = np.where(pd_lend_df['annual_inc'] >= 250000, pd_lend_df['annual_inc'].median(),pd_lend_df['annual_inc'])
fig = px.box(pd_lend_df, y="annual_inc")
fig.update_traces(quartilemethod="linear") # or "inclusive", or "linear" by default
fig.show()


# # Univariate analysis categorical columns

In [ ]:
def plot_histogram_pie_chart_categorical_columns(category_col_name):
    #fig = px.pie(pd_lend_df, names=category_col_name , title = "box plot  " + category_col_name) # Add title to plots
    #fig.show()
    #For now only plotting histograms but box plots can be plotted easily
    fig = px.histogram(pd_lend_df, x= category_col_name, title = "histogram " + category_col_name)
    fig.show()

In [ ]:
cat_cols = pd_lend_df.select_dtypes(include=['category']).columns
print(cat_cols)
for cat_col in cat_cols:
    print("")
    plot_histogram_pie_chart_categorical_columns(cat_col)
    print("")

# # # Rank Frequency plots ordered categorical variables

In [ ]:
def rank_freq_plots_ordered_categorical(column_name):
    df_rank_freq = pd_lend_df[column_name].value_counts(ascending = False).to_frame().reset_index()
    df_rank_freq.rename(columns = {'index' : 'label'},inplace=True)
    df_rank_freq['rank'] = df_rank_freq.index + 1
    fig = px.line(df_rank_freq, x='rank', y=column_name , title='Rank-Freq plot for ' + column_name, log_x=True, log_y=True)
    fig.show()

In [ ]:
rank_freq_plots_ordered_categorical('purpose')
rank_freq_plots_ordered_categorical('addr_state')


# Segmented univariate Analysis

In [ ]:
# Determine impact of a random variable on loan_status to determine thresholds

In [ ]:
charged_off_pct = round(100*(len(pd_lend_df[pd_lend_df['loan_status'] == "Charged Off"])/len(pd_lend_df)),2)
charged_off_pct

In [ ]:
charged_off_pct_upper = 15
charged_off_pct_lower = 14 

In [ ]:
def pivot_segmented_univariate(column_name):
    table1 = pd.pivot_table(pd_lend_df[['loan_status',column_name]], index = column_name ,columns=['loan_status']
                            , aggfunc=np.size)
    table1.columns = table1.columns.astype(str)
    table1['%ChargedOff'] = round(100*(table1['Charged Off']/(table1['Charged Off'] + table1['Fully Paid'])) ,2)
    table1['ChargedOff_pct_diff_upper'] = table1['%ChargedOff'] - charged_off_pct_upper
    table1['ChargedOff_pct_diff_lower'] = table1['%ChargedOff'] - charged_off_pct_lower
    table1.sort_values(by='%ChargedOff', ascending = False)
    return(table1)

In [ ]:
pd_lend_df['random'] = np.random.random((len(pd_lend_df),1))
pd_lend_df['random'] = np.where(pd_lend_df['random']>=0.5,"H","T")
table1 = pivot_segmented_univariate('random')
table1

In [ ]:
# # # # After few trials The impact of random event is around 0.6% , 
# #  hence we keep the upper and lower thresholds  to upper and lower integers



In [ ]:
def plot_segmented_univariate_numeric(num_column_name):
    fig = px.box(pd_lend_df, x= num_column_name,y="loan_status" )
    # target variable is loan_status, for this exercise it is constant, so hardcoded
    fig.update_traces(quartilemethod="linear") # or "inclusive", or "linear" by default
    fig.show()

In [ ]:
df_num = pd_lend_df.select_dtypes(include=[np.number]).columns
for each_numeric_column in df_num:
    plot_segmented_univariate_numeric(each_numeric_column)

In [ ]:
#Segmented univariate analysis on categorical columns
cat_cols = pd_lend_df.select_dtypes(include=['category']).columns
for cat_col in cat_cols:
    if cat_col == 'loan_status':
        continue
    table1 = pivot_segmented_univariate(cat_col)
    print("")
    print("pivot table for loan_status vs  " , cat_col )
    display(table1)
    print( "-------------------------------------------------------------------------------------------")
    print("")


# Convert numeric columns to categorical and perform segmented analysis

In [ ]:
# Post charged off columns are excluded from this
def convert_numeric_column_to_categorical_analyze(numeric_column):
    q=[0, .05, .25, .5, .75, 0.95,1]
    labels = [5,25,50,75,95,100]
    pd_lend_df['binned_' + numeric_column] = pd.qcut(pd_lend_df[numeric_column], q=q, labels=labels)
    table1 = pivot_segmented_univariate('binned_' + numeric_column)
    print("")
    print("pivot table for loan_status vs  " , 'binned_' + numeric_column )
    display(table1)
    print( "-------------------------------------------------------------------------------------------")
    print("")
    pd_lend_df.drop(columns = ['binned_' + numeric_column],inplace = True)
    

In [ ]:
df_num = pd_lend_df.select_dtypes(include=[np.number]).columns
columns_to_exclude = ['pub_rec','inq_last_6mths','mths_since_last_delinq','total_rec_late_fee','recoveries','total_rec_late_fee','collection_recovery_fee','annual_inc_outliar_adjust','delinq_2yrs']
for each_numeric_column in df_num:
    if each_numeric_column in columns_to_exclude:
        continue
    print(each_numeric_column)
    convert_numeric_column_to_categorical_analyze(each_numeric_column)

In [ ]:
# Segmented Univariate analysis : Mean and Std comparison 

In [ ]:
# Maybe this analysis is redundant since we are anyways plotting medians and quartiles
# neverthesess as practice completing this for alll columns

In [ ]:
columns_to_exclude = ['annual_inc_outliar_adjust']
df_num = pd_lend_df.select_dtypes(include=[np.number]).columns
for each_numeric_column in df_num:
    if each_numeric_column in columns_to_exclude:
        continue
    mean_numeric_column = pd_lend_df[each_numeric_column].mean()
    std_numeric_column = pd_lend_df[each_numeric_column].std()
    print("------------------------------------------------------------------------------------")
    print(each_numeric_column , pd_lend_df[each_numeric_column].describe())
    print("After segmentation by loan status")
    print(pd_lend_df.groupby('loan_status')[each_numeric_column].describe()[['count','mean','std','25%','50%','75%']])
    print("")
    print("------------------------------------------------------------------------------------")
    

In [ ]:
def bivariate_pivot_table(column1, column2):
    table1 = pd.pivot_table(pd_lend_df[['loan_status',column1 , column2]], index = [column1 , column2],columns=['loan_status']
                            , aggfunc=np.size)
    table1.columns = table1.columns.astype(str)
    table1['%ChargedOff'] = round(100*(table1['Charged Off']/(table1['Charged Off'] + table1['Fully Paid'])) ,2)
    table1 = table1[table1['%ChargedOff'].notna()]
    table1['ChargedOff_pct_diff_upper'] = table1['%ChargedOff'] - charged_off_pct_upper
    table1['ChargedOff_pct_diff_lower'] = table1['%ChargedOff'] - charged_off_pct_lower
    table1.sort_values(by='%ChargedOff', ascending = False)
    return(table1)


In [ ]:
cat_cols = pd_lend_df.select_dtypes(include=['category']).columns
for cat_col1 in cat_cols:
    for cat_col2 in cat_cols:
        if cat_col1 == 'loan_status' or cat_col1 == cat_col2 or cat_col2 == 'loan_status':
            continue
        table1 = bivariate_pivot_table(cat_col1,cat_col2)
        print("")
        print("pivot table for loan_status vs  " , cat_col1 , cat_col2 )
        display(table1)
        print( "-------------------------------------------------------------------------------------------")
        print("")


In [ ]:
def convert_numeric_column_to_categorical_analyze_bivariate(numeric_column1,numeric_column2):
    q=[0, .05, .25, .5, .75, 0.95,1]
    labels = [5,25,50,75,95,100]
    pd_lend_df['binned_' + numeric_column1] = pd.qcut(pd_lend_df[numeric_column1], q=q, labels=labels)
    pd_lend_df['binned_' + numeric_column2] = pd.qcut(pd_lend_df[numeric_column1], q=q, labels=labels)
    table1 = bivariate_pivot_table('binned_' + numeric_column1,'binned_' + numeric_column2)
    print("")
    print("pivot table for loan_status vs  " , 'binned_' + numeric_column1 , 'binned_' + numeric_column2 )
    display(table1)
    print( "-------------------------------------------------------------------------------------------")
    print("")
    pd_lend_df.drop(columns = ['binned_' + numeric_column1,'binned_' + numeric_column2],inplace = True)


In [ ]:
def convert_numeric_column_to_categorical_analyze_bivariate_existing_categorical(numeric_column):
    q=[0, .05, .25, .5, .75, 0.95,1]
    labels = [5,25,50,75,95,100]
    pd_lend_df['binned_' + numeric_column] = pd.qcut(pd_lend_df[numeric_column], q=q, labels=labels)
    for cat_col in cat_cols:
        print(cat_col)
        if cat_col == 'loan_status':
            continue
        table1 = bivariate_pivot_table('binned_' + numeric_column,cat_col)
        print("")
        print("pivot table for loan_status vs  " , 'binned_' + numeric_column , cat_col , " category")
        display(table1)
        print( "-------------------------------------------------------------------------------------------")
        print("")
    pd_lend_df.drop(columns = ['binned_' + numeric_column],inplace = True)
   

In [ ]:
df_num = pd_lend_df.select_dtypes(include=[np.number]).columns
columns_to_exclude = ['pub_rec','inq_last_6mths','mths_since_last_delinq','total_rec_late_fee','recoveries','total_rec_late_fee','collection_recovery_fee','annual_inc_outliar_adjust','delinq_2yrs']
for each_numeric_column1 in df_num:
    for each_numeric_column2 in df_num:
        if each_numeric_column1 in columns_to_exclude or each_numeric_column1 == each_numeric_column2 or each_numeric_column2 in columns_to_exclude:
            continue
        convert_numeric_column_to_categorical_analyze_bivariate(each_numeric_column1,each_numeric_column2)
    


In [ ]:
df_num = pd_lend_df.select_dtypes(include=[np.number]).columns
print(df_num)
columns_to_exclude = ['pub_rec','inq_last_6mths','mths_since_last_delinq','total_rec_late_fee','recoveries','total_rec_late_fee','collection_recovery_fee','annual_inc_outliar_adjust','delinq_2yrs']
for each_numeric_column in df_num:
    if each_numeric_column in columns_to_exclude:
        continue
    print(" processing " , each_numeric_column)
    convert_numeric_column_to_categorical_analyze_bivariate_existing_categorical(each_numeric_column)
        

# Derived Columns

# convert object to dates
pd_lend_df['issue_d'] = pd.to_datetime(pd_lend_df['issue_d'],format = '%b-%y' )
pd_lend_df['last_pymnt_d'] = pd.to_datetime(pd_lend_df['last_pymnt_d'],format = '%b-%y' )
pd_lend_df['last_credit_pull_d'] = pd.to_datetime(pd_lend_df['last_credit_pull_d'],format = '%b-%y' )
pd_lend_df['earliest_cr_line'] = pd.to_datetime(pd_lend_df['earliest_cr_line'],format = '%b-%y' )
#print(type(pd_lend_df['issue_d']), type(pd_lend_df['last_pymnt_d']))
pd_lend_df['issue_d'].head()
pd_lend_df['last_pymnt_d'].head()
pd_lend_df['last_credit_pull_d'].head()

pd_annual_inc_out = pd_lend_df[pd_lend_df['revol_bal'] >= 60000][['loan_amnt','loan_status','revol_bal','revol_util']].copy()
#pd_annual_inc_out = pd_annual_inc_out[pd_annual_inc_out['loan_status'] == 'Charged Off']
print(pd_annual_inc_out)

pd_annual_inc_out = pd_lend_df[pd_lend_df['revol_util'] >= 75][['loan_amnt','loan_status','revol_bal','revol_util']].copy()
pd_annual_inc_out = pd_annual_inc_out[pd_annual_inc_out['loan_status'] == 'Charged Off']
print(pd_annual_inc_out)

pd_annual_inc_out = pd_lend_df[((pd_lend_df['revol_util'] >= 75) & (pd_lend_df['revol_bal'] >= 60000))][['loan_amnt','loan_status','revol_bal','revol_util']].copy()
#pd_annual_inc_out = pd_annual_inc_out[pd_annual_inc_out['loan_status'] == 'Charged Off']
print(len(pd_annual_inc_out),len(pd_annual_inc_out[pd_annual_inc_out['loan_status'] == 'Charged Off']))

# Understand how sub grade 1,5 are structured
pd_lend_df['sub_grade'] = pd_lend_df['sub_grade'].str[1:]
pd_lend_df['sub_grade'] = pd_lend_df['sub_grade'].str[1:]
table1 = pd.pivot_table(pd_lend_df[['loan_status','sub_grade']], index = 'sub_grade',columns=['loan_status'] , aggfunc=np.size)
table1['%ChargedOff'] = round(100*(table1['Charged Off']/(table1['Charged Off'] + table1['Fully Paid'])) ,2)
table1.sort_values(by='%ChargedOff', ascending = False)

pd_lend_df['Ratio_funded_to_applied'] = pd_lend_df['funded_amnt']/pd_lend_df['loan_amnt']
new_df = pd_lend_df[pd_lend_df['Ratio_funded_to_applied'] <1]
print(len(new_df))
fig = px.box(new_df, x="Ratio_funded_to_applied")
fig.update_traces(quartilemethod="linear") # or "inclusive", or "linear" by default
fig.show()
pd_lend_df['Ratio_funded_to_applied'].describe()

In [ ]:
print(" run complete")